In [ ]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from tensorflow.keras.datasets import reuters
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
raw_data = pd.read_excel('C:/Users/황윤섭/Desktop/raw_data.xlsx')

In [ ]:
print(k.groupby('카테고리').size().reset_index(name = 'count'))

In [ ]:
a=k.groupby('카테고리').get_group('과학/공학').sample(20000)
b=k.groupby('카테고리').get_group('사회').sample(20000)
c=k.groupby('카테고리').get_group('예술/대중문화').sample(20000)
d=k.groupby('카테고리').get_group('인문').sample(20000)
e=k.groupby('카테고리').get_group('종교').sample(20000)

In [ ]:
raw_data = pd.concat([a,b,c,d,e])

In [ ]:
train_data = raw_data.sample(frac=0.7)

In [ ]:
test_data = raw_data.drop(train_data.index)

In [ ]:
train_data['책소개'] = train_data['책소개'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
train_data = train_data.dropna(how = 'any')

In [ ]:
test_data['책소개'] = test_data['책소개'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
test_data = test_data.dropna(how = 'any')

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()

In [ ]:
#train data 전처리 

In [ ]:
X_train = []
for sentence in train_data['책소개']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
y_train=list(train_data['카테고리'])
y_train=[0 if i=='과학/공학' else i for i in y_train]
y_train=[1 if i=='사회' else i for i in y_train]
y_train=[2 if i=='예술/대중문화' else i for i in y_train]
y_train=[3 if i=='인문' else i for i in y_train]
y_train=[4 if i=='종교' else i for i in y_train]     # 재시작하려면 여기서부터 

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 100
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = total_cnt - rare_cnt + 1 # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
print('책소개 최대 길이 :',max(len(l) for l in X_train))
print('책소개 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 250
below_threshold_len(max_len, X_train)

In [ ]:
max_len = 250
X_train = pad_sequences(X_train, maxlen=max_len) 

In [ ]:
y_train = to_categorical(y_train)

In [ ]:
##test 데이터 전처리 

In [ ]:
X_test = []
for sentence in test_data['책소개']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
y_test=list(test_data['카테고리'])
y_test=[0 if i=='과학/공학' else i for i in y_test]
y_test=[1 if i=='사회' else i for i in y_test]
y_test=[2 if i=='예술/대중문화' else i for i in y_test]
y_test=[3 if i=='인문' else i for i in y_test]
y_test=[4 if i=='종교' else i for i in y_test]     # 재시작하려면 여기서부터 

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1]

In [ ]:
X_test = np.delete(X_test, drop_test, axis=0)
y_test = np.delete(y_test, drop_test, axis=0)
print(len(X_test))
print(len(y_test))

In [ ]:
X_test = pad_sequences(X_test, maxlen=max_len) 

In [ ]:
y_test = to_categorical(y_test)

In [ ]:
##RNN(LSTM) 

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 200))
model.add(LSTM(200))
model.add(Dense(5, activation='softmax'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10, callbacks=[es, mc], validation_data=(X_test, y_test))

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
## 2layers MLP

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
def fit_and_evaluate(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(Dense(256, input_shape=(250,), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.1)
    score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
    return score[1]


In [ ]:
score = fit_and_evaluate(X_train, y_train, X_test, y_test)
print(score)